#### Normaliza o texto [transcrições]

In [3]:
from Funcs.tratamentoDeTexto import normalize_text
from Funcs.getPaths import get_Paths
from Funcs.db import get_languange, get_data, update_data

from Funcs.db import new_collumn
new_collumn('videos_data.db','transcribes','transcription_normalizado','TEXT')

paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]
    normal = normalize_text(get_data('videos_data.db','transcribes','transcription',id)[0][1],get_languange('videos_data.db','links',id)[0])
    update_data('videos_data.db','transcribes','transcription_normalizado',id,normal)





Erro ao adicionar coluna: duplicate column name: transcription_normalizado


#### Normaliza o texto [captions]

In [ ]:
from Funcs.tratamentoDeTexto import normalize_text
from Funcs.getPaths import get_Paths
from Funcs.db import get_data, update_data

from Funcs.db import new_collumn
new_collumn('videos_data.db','captions','captions_normalizado','TEXT')

paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]
    frames = get_data('videos_data.db','captions','caption',id)
    for frame in frames:
        #print('id: '+frame[0])
        #print('caption: '+frame[1])
        normal = normalize_text(frame[1],'en')
        update_data('videos_data.db','captions','captions_normalizado',frame[0],normal)
    




